In [1]:
import pandas as pd
import numpy as np
import re

from pymorphy2 import MorphAnalyzer
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from matplotlib import pyplot as plt
from tqdm import tqdm


%matplotlib inline

In [25]:
train_df = pd.read_csv('../dataset/RoboMed/new_train.csv')
test_df = pd.read_csv('../dataset/RoboMed/new_test.csv')

In [3]:
def explain_diagnosis_code(series):
    regex = '([A-Z])(\d\d)(?:.(\d\*?))?'
    prefixes = ['группа_', 'номер_', 'подномер_']
    def get_explanation(string):
        matches = re.findall(regex, string)
        if len(matches) == 0:
            return 'неизвестный_диагноз'
        return [prefix + token for prefix, token in zip(prefixes, matches[0])]
    return series.apply(lambda string: ' '.join(get_explanation(string)))

In [4]:
train_df['Общее состояние'] = train_df['Общее состояние'].fillna('общее_состояние_неизвестно')
train_df['аллергия'] = train_df['аллергия'].fillna('аллергии_неизвестны')
train_df['Анамнез заболевания'] = train_df['Анамнез заболевания'].fillna('нет_анамнеза')
train_df['Внешний осмотр'] = train_df['Внешний осмотр'].fillna('нет_осмотра')
train_df['Типичные_жалобы'] = train_df['Типичные_жалобы'].fillna('типичные_жалобы_неизвестны')
train_df['Типичные_услуги'] = train_df['Типичные_услуги'].fillna('типичные_услги_неизвестны')
train_df['Типичные_источники_рекламы'] = train_df['Типичные_источники_рекламы'].fillna('типичные_источники_рекламы_неизвестны')
train_df['Код_диагноза'] = explain_diagnosis_code(train_df['Код_диагноза'])

test_df['Общее состояние'] = test_df['Общее состояние'].fillna('общее_состояние_неизвестно')
test_df['аллергия'] = test_df['аллергия'].fillna('аллергии_неизвестны')
test_df['Анамнез заболевания'] = test_df['Анамнез заболевания'].fillna('нет_анамнеза')
test_df['Внешний осмотр'] = test_df['Внешний осмотр'].fillna('нет_осмотра')
test_df['Типичные_жалобы'] = test_df['Типичные_жалобы'].fillna('типичные_жалобы_неизвестны')
test_df['Типичные_услуги'] = test_df['Типичные_услуги'].fillna('типичные_услги_неизвестны')
test_df['Типичные_источники_рекламы'] = test_df['Типичные_источники_рекламы'].fillna('типичные_источники_рекламы_неизвестны')
test_df['Код_диагноза'] = explain_diagnosis_code(test_df['Код_диагноза'])

In [5]:
train_df.Revisit.value_counts()

1    234576
0     48510
Name: Revisit, dtype: int64

In [6]:
train_size = train_df.shape[0]
print(train_size)

283086


In [7]:
# train_df = train_df.fillna('')
# test_df = test_df.fillna('')

In [8]:
re.sub(r"([.\!?,'/()])", ' \\1 ', 'Это самое обычное предложение, которое в том числе заканчивается!')

'Это самое обычное предложение ,  которое в том числе заканчивается ! '

In [9]:
train_dataframe, valid_dataframe = train_test_split(train_df, test_size=0.1,
                                                    stratify=train_df.Revisit, random_state=42)

In [10]:
# y_train = train_dataframe.Revisit

In [11]:
# train_dataframe = train_dataframe.sample(frac=1, random_state=42)
# train_dataframe.Revisit = y_train

In [12]:
def concat_features_ft(df, train=True):
    features = (
        df['Код_диагноза'].str.replace('.', '#') + ' • ' +
#         df['Диагноз'] + ' • ' +
        'Возраст_' + df['Возраст'].astype(str) + ' • ' +
        'Пол_' + df['Пол'].astype(str) + ' • ' +
        df['Общее состояние'] + ' • ' +
        df['аллергия'] + ' • ' +
        df['Анамнез заболевания'] + ' • ' +
        df['Внешний осмотр'] + ' • ' +
        df['Типичные_жалобы'] + ' • ' +
        df['Типичные_услуги'] + ' • ' +
        df['Типичные_источники_рекламы']
    )
    
    text = features
    if train:
        text = '__label__' + df.Revisit.astype(str) + ' ' + text

    return text.str.lower().str.replace('\n', '').apply(lambda string: re.sub(r"([.\!?,'/()\-№:;])", ' \\1 ', string))

In [13]:
fasttext_train = concat_features_ft(train_dataframe)
fasttext_valid = concat_features_ft(valid_dataframe)
fasttext_test = concat_features_ft(test_df, train=False)

In [14]:
with open('fasttext_train.txt', 'w') as file:
    for line in fasttext_train:
        file.write(line + '\n')

In [15]:
with open('fasttext_valid.txt', 'w') as file:
    for line in fasttext_valid:
        file.write(line + '\n')

In [16]:
with open('fasttext_test.txt', 'w') as file:
    for line in fasttext_test:
        file.write(line + '\n')

In [123]:
! tail -n 3 fasttext_test.txt

группа_e номер_28 подномер_8 • возраст_23 • пол_2 • общее_состояние_неизвестно • аллергии_неизвестны • пришел на повторный прием по результатам замера ад .  на фоне приема препаратов цифры ад 110 - 120 / 80 мм рт ст ,  пульс 90 - 100 в мин .  магне в6 ,  глицин и пустырник не принимает в течение 10 дней .  • нет_осмотра • типичные_услги_неизвестны • типичные_источники_рекламы_неизвестны
группа_k номер_02 подномер_1 • возраст_61 • пол_2 • общее_состояние_неизвестно • аллергии_неизвестны • нет_анамнеза • удовлетворительное .  • лечение кариеса с наложением пломбы из светоотверждаюшего композита • другое
группа_n номер_86 подномер_ • возраст_28 • пол_2 • общее самочувствие удовлетворительное .  кожные покровы и видимые слизистые обычной окраски .   язык чистый ,  влажный .  живот мягкий ,   безболезненный во всех отделах .  стул ,  мочеиспускание не нарушены .  • не отягощен • нет_анамнеза • нет_осмотра • прием врача - акушера - гинеколога повторный ,  амбулаторный • другое | рекомендации

In [50]:
! wc fasttext_train.txt

   254777   9747764 129880663 fasttext_train.txt


In [61]:
! fasttext fasttext supervised -input fasttext_train.txt -wordNgrams 3 -loss hs -thread 32 -output model_categories -dim 300 -epoch 8 -lr 0.1 -pretrainedVectors ft_native_300_ru_wiki_lenta_lower_case.vec

/bin/sh: 1: fasttext: not found


In [ ]:
! fasttext test model_categories.bin fasttext_valid.txt

In [ ]:
! fasttext predict-prob model_categories.bin fasttext_test.txt > y_pred_test.txt

In [ ]:
! fasttext predict-prob model_categories.bin fasttext_valid.txt > y_pred_valid.txt

In [20]:
def read_pred(filename):
    with open(filename) as file:
        lines = file.readlines()
        y_pred = np.array([float(line.split()[1]) if line.split()[0] == '__label__1' else 1 - float(line.split()[1])
                           for line in lines])
    return y_pred

In [21]:
y_pred_valid = read_pred('y_pred_valid.txt')

In [22]:
roc_auc_score(valid_dataframe.Revisit, y_pred_valid)

0.7033888063631192

In [23]:
y_pred_test = read_pred('y_pred_test.txt')
submission = pd.read_csv('/root/dataset/RoboMed/sample_submission.csv')
submission.proba = y_pred_test
submission.to_csv('submission_ft_all.csv', index=False)

In [24]:
y_pred_test.min()

0.06716900000000003